In [ ]:
!pip install langchain
!pip install pypdf
!pip install numpy

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser, RunnablePassthrough
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import FAISS
from pathlib import Path
import random

In [ ]:
llm = ChatOpenAI()
llm("Erstelle eine Frage über meine Dokumente.")

In [ ]:
prompt = PromptTemplate.from_template(
    "Erstelle eine Frage über meine Dokumente. Die Dokumente haben folgenden Inhalt: {dokument}?"
)

In [ ]:
print(prompt.invoke({"dokument": "Mein Dokument Inhalt"}))

In [ ]:
pipeline = prompt | llm | StrOutputParser()

In [ ]:
pipeline.invoke({"dokument": "Deep Learning (deutsch: mehrschichtiges Lernen, tiefes Lernen oder tiefgehendes Lernen) bezeichnet eine Methode des maschinellen Lernens, die künstliche neuronale Netze (KNN) mit zahlreichen Zwischenschichten (englisch hidden layers) zwischen Eingabeschicht und Ausgabeschicht einsetzt und dadurch eine umfangreiche innere Struktur herausbildet. Es ist eine spezielle Methode der Informationsverarbeitung."})

In [ ]:
pfad = Path("data")
for pdf_datei in pfad.glob("**/*.pdf"):
    print(pdf_datei)

In [ ]:
for pdf_datei in pfad.glob("**/*.pdf"):
    loader = PyPDFLoader("example_data/layout-parser-paper.pdf")
    pages = loader.load_and_split()
    print(f"Die Datei {pdf_datei} hat {len(pages)} Seiten.")

In [ ]:
documents = []

for pdf_datei in pfad.glob("**/*.pdf"):
    loader = PyPDFLoader("example_data/layout-parser-paper.pdf")
    pages = loader.load_and_split()
    print(f"Die Datei {pdf_datei} hat {len(pages)} Seiten.")
    documents.extend(pages)
    
print(f"Wir haben insgesamt {len(documents)} Seiten.")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texte = text_splitter.split_documents(documents)
print(f"Wir haben insgesamt {len(texte)} Texte.")
print(f"Beispieltext 1: {texte[0]}")
print(f"Beispieltext 2: {texte[1]}")

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L12-v2")

In [ ]:
embeddings = embedding_model.embed_documents(texte)
print(f"Wir haben insgesamt {len(embeddings)} Embeddings.")
print(f"Beispieltext 1: {embeddings[0]}")

In [ ]:
embedded_frage = embedding_model.embed_query("Was ist Deep Learning?")
print(embedded_frage)

In [ ]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

similarities = [cosine_similarity(embedded_frage, embedding) for embedding in embeddings]
print(similarities)

In [ ]:
[e for _, e in sorted(zip(similarities, embeddings))]
[texte for _, t in sorted(zip(similarities, texte))]
similarities = sorted(similarities)

In [ ]:
for i in range(10):
    print(f"Ähnlichkeit: {similarities[i]}, Text: {texte[i]}")

In [ ]:
datenbank = FAISS.from_documents(texte, embedding=embedding_model)
datenbank.save_local("data/index.faiss")

In [ ]:
prompt = PromptTemplate.from_template(
    "Erstelle eine einzige Frage über Kontext: {dokument}."
)

In [ ]:
dokument = datenbank[random.randint(0, len(datenbank))]

frage_chain = (
    {"dokument": dokument}
    | prompt
    | llm
    | StrOutputParser()
)

print(f"Frage: {frage_chain.invoke()}")

In [ ]:
print("Bitte beantworte die Frage")
antwort = input()
print(f"Ihre Antwort war:{antwort}")

In [ ]:
prompt2 = PromptTemplate.from_template(
    "Gegeben war folgendes Dokument: {doku}. Die Frage lautete: {frage}. Die Antwort war: {antwort}."
    "Bewerte die Antwort auf einer Skala von 1 bis 10 und sag was eine bessere Antowrt gewesen wäre."
)

korrektur_chain = (
    {"dokument": dokument}
    | prompt2
    | llm
    | StrOutputParser()
)

print(korrektur_chain.invoke())